In [37]:
library(Seurat)
library(Signac)
library(CellChat)
library(patchwork)
library(argparse)
source("/tscc/projects/ps-epigen/users/kdang/eye_final/source_FunFig.R")
library(dplyr)                                                               
library(devtools)
library(hdf5r)
source("/tscc/projects/ps-epigen/bpToolbox/utils.R")
source("/tscc/nfs/home/a2jorgensen/toolbox/BpdToolbox/qc_report.R")
library(reshape2)
library(Nebulosa)
library(stringr)
library(harmony)

In [38]:
setwd("/tscc/projects/ps-epigen/users/a2jorgensen/sandbox/250225_humanAT2_organoids/data/Clustering/JB_1466/")

In [39]:
sobj<- readRDS("JB_1466_w_demux_cond_&_day.RDS")

In [40]:
sobj

An object of class Seurat 
44696 features across 16556 samples within 2 assays 
Active assay: SCT (21631 features, 5000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: RNA
 3 dimensional reductions calculated: pca, umap.rna, harm.pca

In [41]:
unique(colnames(sobj@meta.data))

[1] "orig.ident"         "nCount_RNA"         "nFeature_RNA"      
 [4] "percent.mt"         "nCount_SCT"         "nFeature_SCT"      
 [7] "SCT_snn_res.1"      "seurat_clusters"    "INT_ID"            
[10] "NUM.SNPS"           "NUM.READS"          "DROPLET.TYPE"      
[13] "BEST.GUESS"         "BEST.LLK"           "NEXT.GUESS"        
[16] "NEXT.LLK"           "DIFF.LLK.BEST.NEXT" "BEST.POSTERIOR"    
[19] "SNG.POSTERIOR"      "SNG.BEST.GUESS"     "SNG.BEST.LLK"      
[22] "SNG.NEXT.GUESS"     "SNG.NEXT.LLK"       "SNG.ONLY.POSTERIOR"
[25] "DBL.BEST.GUESS"     "DBL.BEST.LLK"       "DIFF.LLK.SNG.DBL"  
[28] "Day"                "cond.day"           "condition"

In [42]:
Idents(sobj) <- "seurat_clusters"

In [43]:
sobj <- RenameIdents(object = sobj, 
                     "0" = "Mature AT2", 
                     "1" = "Aberrant Basal", 
                     "2" = "Mature AT2", 
                     "3" = "Mature AT2", 
                     "4" = "Fresh AT2", 
                     "5" = "Mature AT2", 
                     "6" = "Mature AT2", 
                     "7" = "Mature AT2", 
                     "8" = "Aberrant Basal", 
                     "9" = "Aberrant Basal", 
                     "10" = "Fresh AT2", 
                     "11" = "Aberrant Basal", 
                     "12" = "Mature AT2", 
                     "13" = "Mature AT2", 
                     "14" = "Mature AT2", 
                     "15" = "Mature AT2", 
                     "16" = "Aberrant Basal", 
                     "17" = "Mature AT2", 
                     "18" = "Mature AT2", 
                     "19" = "Mature AT2", 
                     "20" = "Mature AT2", 
                     "21" = "Fresh AT2", 
                     "22" = "Fresh AT2", 
                     "23" = "Aberrant Basal", 
                     "24" = "Mature AT2")


In [44]:
unique(Idents(sobj))

[1] Mature AT2     Aberrant Basal Fresh AT2     
Levels: Mature AT2 Aberrant Basal Fresh AT2

In [45]:
sobj <- StashIdent(object = sobj, save.name = "broad_anno")

In [46]:
unique(colnames(sobj@meta.data))

[1] "orig.ident"         "nCount_RNA"         "nFeature_RNA"      
 [4] "percent.mt"         "nCount_SCT"         "nFeature_SCT"      
 [7] "SCT_snn_res.1"      "seurat_clusters"    "INT_ID"            
[10] "NUM.SNPS"           "NUM.READS"          "DROPLET.TYPE"      
[13] "BEST.GUESS"         "BEST.LLK"           "NEXT.GUESS"        
[16] "NEXT.LLK"           "DIFF.LLK.BEST.NEXT" "BEST.POSTERIOR"    
[19] "SNG.POSTERIOR"      "SNG.BEST.GUESS"     "SNG.BEST.LLK"      
[22] "SNG.NEXT.GUESS"     "SNG.NEXT.LLK"       "SNG.ONLY.POSTERIOR"
[25] "DBL.BEST.GUESS"     "DBL.BEST.LLK"       "DIFF.LLK.SNG.DBL"  
[28] "Day"                "cond.day"           "condition"         
[31] "broad_anno"

In [47]:
sobj.0 <- subset(sobj, subset = Day %in% "Day.0")
sobj.4 <- subset(sobj, subset = Day %in% "Day.4")
sobj.7 <- subset(sobj, subset = Day %in% "Day.7")

In [51]:
norm_cellquant_bplot2.0 <- function (sobj, dset.col = "orig.ident", xlab = "seurat_clusters", stack.by = "condition", rand.cols = FALSE){
    # Create table of cells observed for our dataset and xaxis variable
    table <- table(sobj@meta.data[[dset.col]], sobj@meta.data[[xlab]])
    
    # Normalize the table by dataset cell quantity
    nmax <- max(rowSums(table))
    norm.df <- data.frame(row.names = rownames(table))
    for (r in seq(1, nrow(table))) {
        rsum <- sum(table[r, ])
        cfactor <- nmax / rsum
        trow <- table[r, ] * cfactor
        trow <- trow / sum(trow) * 100
        norm.df <- rbind(norm.df, trow)
    }
    
    colnames(norm.df) <- colnames(table)
    rownames(norm.df) <- rownames(table)
    
    # Add the stack.by variable
    vars <- c()
    for (r in rownames(norm.df)){
        var <- sobj@meta.data[sobj@meta.data[[dset.col]] == r, stack.by][1]
        vars <- append(vars, var)
    }
    norm.df[[stack.by]] <- vars
    
    # Melt dataframe for ggplot
    norm.df.m <- melt(norm.df, id = stack.by)
    
    # Create ggplot
    cols <- c("cadetblue4", "lightgoldenrod", "salmon", "paleturquoise3", "palegreen3", "mediumpurple1", 
              "salmon", "lightblue4", "navajowhite1", "magenta", "coral2", "mediumorchid1", "midnightblue", 
              "lightgoldenrodyellow", "black", "lightgrey", "mistyrose4", "darkcyan", "steelblue2", 
              "darkolivegreen3", "mediumpurple1", "lightskyblue", "firebrick2", "burlywood", "chartreuse1", 
              "deeppink2", "khaki", "powderblue", "slategrey", "springgreen", "yellow3", "orange2", 
              "lightsteelblue3", "tomato3", "palegreen4", "grey27", "darkseagreen", "blue", "darkorchid", 
              "snow2", "peachpuff2", "magenta2", "yellowgreen", "cornflowerblue", "chocolate", "blueviolet", 
              "lightblue1", "plum2", "gold", "orangered", "darkslateblue", "aquamarine3", "mediumseagreen", 
              "lightsalmon", "darkgoldenrod", "peru", "lavender", "turquoise", "deeppink", "tomato", "plum", 
              "seashell", "springgreen4", "lightyellow", "chartreuse3", "royalblue", "lightpink", "lightgreen", 
              "seashell2", "bisque", "lightcoral", "forestgreen", "slateblue", "darkkhaki", "darkslategray", 
              "darkgray", "hotpink", "mediumvioletred", "dodgerblue", "indianred1", "yellow2", "greenyellow", 
              "darkturquoise", "darkred", "mediumslateblue", "#DC143C", "goldenrod", "green4", "powderblue", 
              "antiquewhite", "lightseagreen", "darkviolet", "mistyrose", "thistle", "wheat", "firebrick", 
              "forestgreen", "mediumorchid", "lightskyblue2", "slategray", "lightsteelblue", "cornflower", 
              "indigo", "darkviolet", "burlywood4", "mediumaquamarine", "fuchsia", "chartreuse", "gainsboro", 
              "ivory", "tan", "oldlace")
   
    # Randomize colors for fun
    if (rand.cols == TRUE){
        cols <- sample(x = cols, size = ncol(norm.df), replace = F)
    }
    
    # Create ggplot
    bp <- ggplot(norm.df.m, aes(fill = .data[[stack.by]], y = value, x = variable)) + 
        geom_bar(position="fill", stat="identity") + 
        scale_fill_manual(values = cols) +
        theme(axis.text.x = element_text(angle = 40, vjust = 0.5, hjust=1)) + 
        ggtitle("Normalized Stacked Barplot") + 
        xlab(xlab) + 
        ylab("pct.cell.quantity")
    
    return(list(plot = bp, summary_table = norm.df))
}


In [68]:
result<- norm_cellquant_bplot2.0(sobj.0, dset.col = "SNG.BEST.GUESS", xlab = "broad_anno", stack.by = "broad_anno", rand.cols = TRUE)

plot <- result$plot
summary_table <- result$summary_table
print("Day 0 -- ZBC1 = Control samples at day 0 ZBI1 = IPF samples at day 0")
summary_table
result<- norm_cellquant_bplot2.0(sobj.4, dset.col = "SNG.BEST.GUESS", xlab = "broad_anno", stack.by = "broad_anno", rand.cols = TRUE)

plot <- result$plot
summary_table <- result$summary_table
print("Day 4 -- ZBC2 = Control samples at day 4 ZBI3 = IPF samples at day 4 ")
summary_table

result<- norm_cellquant_bplot2.0(sobj.7, dset.col = "SNG.BEST.GUESS", xlab = "broad_anno", stack.by = "broad_anno", rand.cols = TRUE)

plot <- result$plot
summary_table <- result$summary_table
print("Day 7 -- ZBC3 =  Control samples at day 7 ZBI2 = IPF samples at day 7")
summary_table




[1] "Day 0 -- ZBC1 = Control samples at day 0 ZBI1 = IPF samples at day 0"


,Mature AT2,Aberrant Basal,Fresh AT2,broad_anno
,<dbl>,<dbl>,<dbl>,<fct>
0_0_1_ZBC1,37.867888,11.8378025,50.29431,Mature AT2
0_0_4_ZBl1,5.869797,0.1067236,94.02348,Fresh AT2


[1] "Day 4 -- ZBC2 = Control samples at day 4 ZBI3 = IPF samples at day 4 "


,Mature AT2,Aberrant Basal,Fresh AT2,broad_anno
,<dbl>,<dbl>,<dbl>,<fct>
0_0_2_ZBC2,87.282835,6.254343,6.462821,Mature AT2
0_0_6_ZBl3,1.890034,92.869416,5.240550,Aberrant Basal


[1] "Day 7 -- ZBC3 =  Control samples at day 7 ZBI2 = IPF samples at day 7"


,Mature AT2,Aberrant Basal,Fresh AT2,broad_anno
,<dbl>,<dbl>,<dbl>,<fct>
0_0_3_ZBC3,86.44216,11.73010,1.827742,Mature AT2
0_0_5_ZBl2,4.61165,90.38835,5.000000,Aberrant Basal


In [62]:
table(sobj.0$SNG.BEST.GUESS)
table(sobj.4$SNG.BEST.GUESS)
table(sobj.7$SNG.BEST.GUESS)


0_0_1_ZBC1 0_0_4_ZBl1 
      1529        937 


0_0_2_ZBC2 0_0_6_ZBl3 
      2878       1164 


0_0_3_ZBC3 0_0_5_ZBl2 
      7988       2060 

In [ ]:
colors<- read.csv("../colors_CIRM.csv", header = TRUE, sep = ",", stringsAsFactors = FALSE)

In [ ]:
colors<- data_frame(c(""))

In [ ]:
colors

In [ ]:
unique(sobj$broad_anno)

In [ ]:
colors <- data.frame(
  cell_type = c("Aberrant Basal", "Mature AT2", "Fresh AT2"),
  color = c("#00CD00", "#4169E1", "#63B8FF")
)


In [ ]:
options(repr.plot.width=9, repr.plot.height=8)
umap1<- DimPlot(sobj, reduction = "umap.rna", group.by = "broad_anno") + NoLegend() + scale_color_manual(values = setNames(colors$color, colors$cell_type))


In [ ]:
umap1

In [ ]:
 pdf("unlabeled.broad.anno.UMAP.pdf", width = 9, height = 8)
 print(umap1)
 dev.off()

In [ ]:
9/8

In [ ]:
colors <- data.frame(
  cell_type = c("IPF", "control"),
  color = c("#A020F0", "#CD9B1D" )
)

In [ ]:
options(repr.plot.width=20, repr.plot.height=8)
umap2<- DimPlot(sobj, reduction = "umap.rna", group.by = "condition", split.by = "Day", ncol = 3) + scale_color_manual(values = setNames(colors$color, colors$cell_type))

In [ ]:
 pdf("split.by.day.condition.UMAP.pdf", width = 20, height = 8)
 print(umap2)
 dev.off()

In [ ]:
umap2

In [ ]:
20/8

In [ ]:
8/20

In [ ]:
options(repr.plot.width=20, repr.plot.height=8)
# Load the necessary library
library(cowplot)

# Combine the plots side by side
plot_grid(umap1, umap2, ncol = 2)


In [ ]:
options(repr.plot.width=30, repr.plot.height=8)

# Load necessary libraries
library(ggplot2)
library(gridExtra)
library(cowplot)

colors <- data.frame(
  cell_type = c("Aberrant Basal", "Mature AT2", "Fresh AT2"),
  color = c("#00CD00", "#4169E1", "#63B8FF")
)

# Plot 1 with locked aspect ratio
umap1 <- DimPlot(sobj, reduction = "umap.rna", group.by = "broad_anno") + 
  NoLegend() + 
  scale_color_manual(values = setNames(colors$color, colors$cell_type)) + 
  coord_fixed(ratio = 1.125)  # Lock the aspect ratio


colors <- data.frame(
  cell_type = c("IPF", "control"),
  color = c("#A020F0", "#CD9B1D" )
)

# Plot 2 with locked aspect ratio
umap2 <- DimPlot(sobj, reduction = "umap.rna", group.by = "condition", split.by = "Day", ncol = 3) + 
  NoLegend() + 
  scale_color_manual(values = setNames(colors$color, colors$cell_type)) + 
  coord_fixed(ratio = 1)  # Lock the aspect ratio

# Combine the two plots side by side while preserving aspect ratio
grid.arrange(umap1, umap2, ncol = 2)


In [ ]:
colors<- read.table(file = "../colors_CIRM.csv")